# Modelagem

In [1]:
import xgboost as xgb # Necessária para utilizar o algoritmo XGBoost
from sklearn.metrics import mean_squared_error # A métrica de avaliação dos modelos
from sklearn.model_selection import TimeSeriesSplit # Divisão dos dados de Treino/Validação no modelo de Séries Temporais
from lightgbm import LGBMRegressor # Necessária para utilizar o algoritmo Light GBM
from tqdm import  # Monitoramento de execução de loops
import numpy as np
import pandas as pd
from utils import * 

In [2]:
df_centroides = pd.read_csv('df_centroides.csv')

In [3]:
df_centroides.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 40 columns):
C0                            50 non-null float64
C1                            50 non-null float64
C2                            50 non-null float64
C0_COD_CICLO_ANO              53 non-null int64
C0_COD_CICLO                  53 non-null int64
C0_FLG_DATA_mean              53 non-null int64
C0_FLG_CAMPANHA_MKT_A_mean    53 non-null float64
C0_FLG_CAMPANHA_MKT_B_mean    53 non-null float64
C0_FLG_CAMPANHA_MKT_C_mean    53 non-null float64
C0_FLG_CAMPANHA_MKT_D_mean    53 non-null float64
C0_PCT_DESCONTO_mean          53 non-null float64
C0_PCT_DESCONTO_min           53 non-null float64
C0_PCT_DESCONTO_max           53 non-null float64
C0_VL_PRECO_mean              53 non-null float64
C0_VL_PRECO_min               53 non-null float64
C0_VL_PRECO_max               53 non-null float64
C2_COD_CICLO_ANO              53 non-null int64
C2_COD_CICLO                  53 non-null int64
C2_FLG_

In [4]:
# Performance dos Algoritmos (com parâmetros otimizados) em cada grupo
performances = {}

# Algoritmos (com parâmetros otimizados) treinados no conjunto de Treino
modelos_treinados = {}

for i in tqdm(['C0','C1','C2']):
    #Selecionando os 50 data points para treino e validação
    df_aux = df_centroides.loc[:49,df_centroides.columns.str.contains(i)]
    # Definindo base de treino 
    X_train = df_aux[:-5].drop(i,axis=1)
    y_train = df_aux[:-5][i]
    # Definindo base de validação
    X_test = df_aux[-5:].drop(i,axis=1)
    y_test = df_aux[-5:][i]
    
    # Aplicando optuna para otimização dos parâmetros do LGBM
    study_lgbm = optuna.create_study(direction="minimize", study_name="LGBM Regressor")
    func_lgbm = lambda trial: objective_lgbm(trial, X_train, y_train)
    study_lgbm.optimize(func_lgbm, n_trials=20)
    best_lgbm = LGBMRegressor(**study_lgbm.best_params).fit(X_train,y_train)
    
    # Aplicando optuna para otimização dos parâmetros do XGBoost
    study_xgb = optuna.create_study(direction="minimize", study_name="XGB Regressor")
    func_xgb = lambda trial: objective_xgb(trial, X_train, y_train)
    study_xgb.optimize(func_xgb, n_trials=20)
    best_xgb = xgb.XGBRegressor(**study_xgb.best_params).fit(X_train, y_train)
    
    # Armazendando melhores modelos de cada grupo
    modelos_treinados[i] = [best_lgbm,best_xgb]
    
    # Previsão das amostras de Validacao
    y_pred_lgbm = best_lgbm.predict(X_test)                         
    y_pred_xgb = best_xgb.predict(X_test)    
    
    # Analise da performace dos algoritmos
    performances[i] = [('lgbm',mean_squared_error(y_test, y_pred_lgbm,squared=False)),('xgb',mean_squared_error(y_test, y_pred_xgb,squared=False))]   

  0%|                                                                                            | 0/3 [00:00<?, ?it/s][I 2023-02-17 21:26:30,060] A new study created in memory with name: LGBM Regressor
C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\optuna\distributions.py:704: UserWarning: The distribution is specified by [0.2, 0.95] and step=0.1, but the range is not divisible by `step`. It will be replaced by [0.2, 0.9].
  low=low, old_high=old_high, high=high, step=step
C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[1]	valid_0's rmse: 237097	valid_0's l2: 5.62149e+10
[2]	valid_0's rmse: 237097	valid_0's l2: 5.62149e+10
[3]	valid_0's rmse: 237097	valid_0's l2: 5.62149e+10
[4]	valid_0's rmse: 237097	valid_0's l2: 5.62149e+10
[5]	valid_0's rmse: 237097	valid_0's l2: 5.62149e+10
[6]	valid_0's rmse: 237097	valid_0's l2: 5.62149e+10
[7]	valid_0's rmse: 237097	valid_0's l2: 5.62149e+10
[8]	valid_0's rmse: 237097	valid_0's l2: 5.62149e+10
[9]	valid_0's rmse: 237097	valid_0's l2: 5.62149e+10
[10]	valid_0's rmse: 237097	valid_0's l2: 5.62149e+10
[11]	valid_0's rmse: 237097	valid_0's l2: 5.62149e+10


C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[1]	valid_0's rmse: 252632	valid_0's l2: 6.3823e+10
[2]	valid_0's rmse: 252632	valid_0's l2: 6.3823e+10
[3]	valid_0's rmse: 252632	valid_0's l2: 6.3823e+10
[4]	valid_0's rmse: 252632	valid_0's l2: 6.3823e+10
[5]	valid_0's rmse: 252632	valid_0's l2: 6.3823e+10
[6]	valid_0's rmse: 252632	valid_0's l2: 6.3823e+10
[7]	valid_0's rmse: 252632	valid_0's l2: 6.3823e+10
[8]	valid_0's rmse: 252632	valid_0's l2: 6.3823e+10
[9]	valid_0's rmse: 252632	valid_0's l2: 6.3823e+10
[10]	valid_0's rmse: 252632	valid_0's l2: 6.3823e+10
[11]	valid_0's rmse: 252632	valid_0's l2: 6.3823e+10


C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


...

C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[21:27:10] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:21480.3
Will train until validation_0-rmse hasn't improved in 10 rounds.
[1]	validation_0-rmse:18065.5
[2]	validation_0-rmse:15266
[3]	validation_0-rmse:14744.9
[4]	validation_0-rmse:12863.7
[5]	validation_0-rmse:11889.6
[6]	validation_0-rmse:10813.2
[7]	validation_0-rmse:10623.2
[8]	validation_0-rmse:9883.26
[9]	validation_0-rmse:9279.09
[10]	validation_0-rmse:8873.35
[11]	validation_0-rmse:8561.05
[12]	validation_0-rmse:8319.15
[13]	validation_0-rmse:8123.54
[14]	validation_0-rmse:8013.21
[15]	validation_0-rmse:7902.66
[16]	validation_0-rmse:7828.02
[17]	validation_0-rmse:7779.03
[18]	validation_0-rmse:7725.1
[19]	validation_0-rmse:7686.19
[20]	validation_0-rmse:7675.96
[21]	validation_0-rmse:7584.46
[22]	validation_0-rmse:7542.27
[23]	validation_0-rmse:7521.81
[24]	validation_0-rmse:7504.85
[25]	vali

[I 2023-02-17 21:27:10,565] Trial 19 finished with value: 14882.823096762779 and parameters: {'n_estimators': 690, 'learning_rate': 0.1539173759555016, 'num_leaves': 8, 'max_depth': 5, 'feature_fraction': 0.30000000000000004}. Best is trial 7 with value: 14570.52898199698.
C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[21:27:10] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:40<00:00, 13.58s/it]


Como podemos ver, analisando o RMSE de ambos os modelos aplicados ao conjunto de validação nos três grupos, identificamos que para o Grupo0 e o Grupo1 o XGBoost obteve melhor perfomance no conjunto de validação do que o LGBM. Porém, para o Grupo2, o LGBM performou melhor.

In [5]:
performances

{'C0': [('lgbm', 38583.90182431084), ('xgb', 33675.68992942772)],
 'C1': [('lgbm', 567924.187390963), ('xgb', 414692.29107130726)],
 'C2': [('lgbm', 15068.743445751657), ('xgb', 21212.665536267355)]}

Treinando o melhor modelo para o Grupo0 em todo o conjunto de Treino+Validação

In [6]:
X_C0_train = df_centroides.loc[:49,df_centroides.columns.str.contains('C0')].drop('C0',axis=1)

y_C0_train = df_centroides.loc[:49,df_centroides.columns.str.contains('C0')]['C0']

final_model_C0 = modelos_treinados['C0'][1].fit(X_C0_train,y_C0_train)

C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[21:28:12] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Treinando o melhor modelo para o Grupo1 em todo o conjunto de Treino+Validação

In [7]:
X_C1_train = df_centroides.loc[:49,df_centroides.columns.str.contains('C1')].drop('C1',axis=1)
X_C1_test = df_centroides.loc[50:,df_centroides.columns.str.contains('C1')].drop('C1',axis=1)

y_C1_train = df_centroides.loc[:49,df_centroides.columns.str.contains('C1')]['C1']

final_model_C1 = modelos_treinados['C1'][1].fit(X_C1_train,y_C1_train)
final_pred_C1 = final_model_C1.predict(X_C1_test)

X_C1_test['C1'] = final_pred_C1

[21:28:13] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Treinando o melhor modelo para o Grupo2 em todo o conjunto de Treino+Validação

In [8]:
X_C2_train = df_centroides.loc[:49,df_centroides.columns.str.contains('C2')].drop('C2',axis=1)
X_C2_test = df_centroides.loc[50:,df_centroides.columns.str.contains('C2')].drop('C2',axis=1)

y_C2_train = df_centroides.loc[:49,df_centroides.columns.str.contains('C2')]['C2']

final_model_C2 = modelos_treinados['C2'][0].fit(X_C2_train,y_C2_train)
final_pred_C2 = final_model_C2.predict(X_C2_test)

X_C2_test['C2'] = final_pred_C2

[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4


Salvando melhores modelos nos três grupos com amostras dos ciclos 201801 ao ciclo 202015

In [9]:
import pickle
pickle.dump(final_model_C0, open('best_model_C0.pkl', 'wb'))
pickle.dump(final_model_C1, open('best_model_C1.pkl', 'wb'))
pickle.dump(final_model_C2, open('best_model_C2.pkl', 'wb'))